In [10]:
from accelerate import Accelerator
from preprocess import prepare_dataset
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch, os
from PIL import Image
from datasets import load_dataset

In [11]:
RAW_DATA_DIR = 'data/raw'
ROOT = 'data'
IMG_DIM = 500

# accelerator = Accelerator()

prepare_dataset(RAW_DATA_DIR, ROOT, IMG_DIM, split=False)

In [1]:
!accelerate launch --mixed_precision="fp16"  train.py \
    --pretrained_model_name_or_path="stablediffusionapi/dreamshaper-v7" \
    --instance_data_dir="C:/Users/Xin Yu/Desktop/data_science/stable-diffusion-ft-w-lora/data/train" \
    --instance_prompt="Milk tea the cat" \
    --dataloader_num_workers=0 \
    --resolution=500 \
    --center_crop \
    --train_batch_size=2 \
    --gradient_accumulation_steps=4 \
    --max_train_steps=500 \
    --learning_rate=1e-04 \
    --max_grad_norm=1 \
    --lr_scheduler="cosine" --lr_warmup_steps=0 \
    --output_dir="C:/Users/Xin Yu/Desktop/data_science/stable-diffusion-ft-w-lora/output/500" \
    --checkpointing_steps=500 \
    --validation_prompt="Milk tea the cat" \
    --validation_epochs=100 \
    --seed=1337

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'
07/04/2023 01:53:11 - INFO - __main__ - Distributed environment: NO
Num processes: 1
Process index: 0
Local process index: 0
Device: cuda
Mixed precision type: fp16

You are using a model of type clip_text_model to instantiate a model of type . This is not supported for all configurations of models and can yield errors.
{'variance_type'} was not found in config. Values will be initialized to default values.
{'num_attention_heads'} was not found in config. Values will be initialized to default values.
07/04/2023 01:53:16 - INFO - __main__ - ***** Running training *****
07/04/2023 01:53:16 - INFO - __main__ -   Num examples = 62
07/04/2023 01:53:16 - INFO - __main__ -   Num batches each epoch = 31
07/04/2023 01:53:16 - INFO - __main__ -   Num Epochs = 63
07/04/2023 01:53:16 - INFO - __main__ -   Instantaneous batch size per device = 2
07/04/2023 01:53:16 - INFO - __main_

In [1]:
import torch
from diffusers import StableDiffusionPipeline, DPMSolverMultistepScheduler

torch.cuda.empty_cache()
pipe = StableDiffusionPipeline.from_pretrained("dreamlike-art/dreamlike-photoreal-2.0", torch_dtype=torch.float16, safety_checker=None)
pipe.scheduler = DPMSolverMultistepScheduler.from_config(pipe.scheduler.config)
pipe.load_lora_weights('output/500/pytorch_lora_weights.bin')
pipe.to("cuda")

A matching Triton is not available, some optimizations will not be enabled.
Error caught was: No module named 'triton'


StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.18.0.dev0",
  "feature_extractor": [
    null,
    null
  ],
  "requires_safety_checker": null,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "DPMSolverMultistepScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [19]:
torch.cuda.empty_cache()
prompt = 'Milk tea the cat, outdoors, detailed, photography, animal, professional, hyperrealistic'
n_prompts = ("extra fingers, extra arms, extra legs, distorted face, mutated, bad anatomy, inaccurate limb, disfigured, ugly hands, bad composition, inaccurate eyes, extra digits, disfigured, extra ears, extra tails, extra legs, multiple face")
images = pipe(prompt, negative_prompt=n_prompts, num_inference_steps=256, guidance_scale=7.5, cross_attention_kwargs={"scale": 1}, num_images_per_prompt=1,).images
[image.show() for image in images]

100%|██████████| 256/256 [01:45<00:00,  2.42it/s]


[None]